# Modelo multinível

[Fonte: storopoli](https://storopoli.github.io/Bayesian-Julia/pages/11_multilevel_models/)

In [1]:
# Pacotes necessários
import Pkg
#Pkg.add("StatsPlots")
#Pkg.add("Distributions")
#Pkg.add("LaTeXStrings")
#Pkg.add("DataFrames")
#Pkg.add("CSV")
#Pkg.add("Plots")

In [2]:
# Carregar pacotes
using StatsPlots, Distributions, LaTeXStrings, DataFrames, CSV

In [3]:
using DataFrames, CSV, HTTP

url = "https://raw.githubusercontent.com/storopoli/Bayesian-Julia/master/datasets/cheese.csv"
cheese = CSV.read(HTTP.get(url).body, DataFrame)
println(cheese)
describe(cheese)

160×4 DataFrame
 Row │ cheese   rater  background  y     
     │ String1  Int64  String7     Int64 
─────┼───────────────────────────────────
   1 │ A            1  rural          67
   2 │ A            1  rural          66
   3 │ B            1  rural          51
   4 │ B            1  rural          53
   5 │ C            1  rural          75
   6 │ C            1  rural          70
   7 │ D            1  rural          68
   8 │ D            1  rural          66
   9 │ A            2  rural          76
  10 │ A            2  rural          76
  11 │ B            2  rural          56
  12 │ B            2  rural          65
  13 │ C            2  rural          82
  14 │ C            2  rural          82
  15 │ D            2  rural          81
  16 │ D            2  rural          77
  17 │ A            3  rural          80
  18 │ A            3  rural          84
  19 │ B            3  rural          71
  20 │ B            3  rural          67
  21 │ C            3  rural          

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,cheese,,A,,D,0,String1
2,rater,5.5,1,5.5,10,0,Int64
3,background,,rural,,urban,0,String7
4,y,70.8438,33,71.5,91,0,Int64


In [4]:
for c in unique(cheese[:, :cheese])
    cheese[:, "cheese_$c"] = ifelse.(cheese[:, :cheese] .== c, 1, 0)
end

cheese[:, :background_int] = map(cheese[:, :background]) do b
    b == "rural" ? 1 :
    b == "urban" ? 2 : missing
end

first(cheese, 5)

,cheese,rater,background,y,cheese_A,cheese_B,cheese_C,cheese_D,background_int
,String1,Int64,String7,Int64,Int64,Int64,Int64,Int64,Int64
1,A,1,rural,67,1,0,0,0,1
2,A,1,rural,66,1,0,0,0,1
3,B,1,rural,51,0,1,0,0,1
4,B,1,rural,53,0,1,0,0,1
5,C,1,rural,75,0,0,1,0,1


In [6]:
X = Matrix(select(cheese, Between(:cheese_A, :cheese_D)));
y = cheese[:, :y];
idx = cheese[:, :background_int];

## Modelos

In [8]:
# Instalar pacotes necessários
#Pkg.add("Turing")
#Pkg.add("StatsBase")

In [9]:
using Turing
using LinearAlgebra: I
using Statistics: mean, std
using Random: seed!
seed!(123)

Random.TaskLocalRNG()

### 1 - Modelo de interceptação aleatória

In [10]:
# varying_intercept
@model function varying_intercept(X, idx, y; n_gr=length(unique(idx)), predictors=size(X, 2))
    #priors
    α ~ Normal(mean(y), 2.5 * std(y))       # population-level intercept
    β ~ filldist(Normal(0, 2), predictors)  # population-level coefficients
    σ ~ Exponential(1 / std(y))             # residual SD
    #prior for variance of random intercepts
    #usually requires thoughtful specification
    τ ~ truncated(Cauchy(0, 2); lower=0)    # group-level SDs intercepts
    αⱼ ~ filldist(Normal(0, τ), n_gr)       # group-level intercepts

    #likelihood
    ŷ = α .+ X * β .+ αⱼ[idx]
    y ~ MvNormal(ŷ, σ^2 * I)
end;

In [11]:
# varying_intercept
model_intercept = varying_intercept(X, idx, y)
chain_intercept = sample(model_intercept, NUTS(), MCMCThreads(), 1_000, 4)
summarystats(chain_intercept) |> DataFrame |> println

┌ Warning: Only a single thread available: MCMC chains are not sampled in parallel
└ @ AbstractMCMC C:\Users\99836932\.julia\packages\AbstractMCMC\fnRmh\src\sample.jl:291
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC C:\Users\99836932\.julia\packages\AdvancedHMC\iWHPQ\src\hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC C:\Users\99836932\.julia\packages\AdvancedHMC\iWHPQ\src\hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC C:\Users\99836932\.julia\packages\AdvancedHMC\iWHPQ\src\hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC

9×8 DataFrame
 Row │ parameters  mean       std       naive_se    mcse        ess       rhat      ess_per_sec 
     │ Symbol      Float64    Float64   Float64     Float64     Float64   Float64   Float64     
─────┼──────────────────────────────────────────────────────────────────────────────────────────
   1 │ α            70.8578   6.05178   0.0956871   0.23822      651.344  1.00389       23.3031
   2 │ β[1]          3.20605  1.26977   0.0200769   0.0305339   2024.44   0.999996      72.4281
   3 │ β[2]        -11.6263   1.26302   0.0199701   0.0304593   2025.53   1.0004        72.4672
   4 │ β[3]          7.1691   1.29857   0.0205323   0.0328251   2024.68   1.00013       72.4367
   5 │ β[4]          1.21296  1.25761   0.0198845   0.0317193   2159.03   1.00003       77.2434
   6 │ σ             6.00355  0.278311  0.00440048  0.00469695  3222.51   0.999358     115.291
   7 │ τ             6.84391  8.48557   0.134169    0.249498     824.663  1.003         29.5039
   8 │ αⱼ[1]        -3.6

### 2 - Modelo de inclinação aleatória

In [12]:
# varying_slope
@model function varying_slope(X, idx, y; n_gr=length(unique(idx)), predictors=size(X, 2))
    #priors
    α ~ Normal(mean(y), 2.5 * std(y))                    # population-level intercept
    σ ~ Exponential(1 / std(y))                          # residual SD
    #prior for variance of random slopes
    #usually requires thoughtful specification
    τ ~ filldist(truncated(Cauchy(0, 2); lower=0), n_gr) # group-level slopes SDs
    βⱼ ~ filldist(Normal(0, 1), predictors, n_gr)        # group-level standard normal slopes

    #likelihood
    ŷ = α .+ X * βⱼ * τ
    y ~ MvNormal(ŷ, σ^2 * I)
end;

In [13]:
model_slope = varying_slope(X, idx, y)
chain_slope = sample(model_slope, NUTS(), MCMCThreads(), 1_000, 4)
summarystats(chain_slope) |> DataFrame |> println

┌ Warning: Only a single thread available: MCMC chains are not sampled in parallel
└ @ AbstractMCMC C:\Users\99836932\.julia\packages\AbstractMCMC\fnRmh\src\sample.jl:291
┌ Info: Found initial step size
│   ϵ = 0.0125
└ @ Turing.Inference C:\Users\99836932\.julia\packages\Turing\szPqN\src\inference\hmc.jl:191
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC C:\Users\99836932\.julia\packages\AdvancedHMC\iWHPQ\src\hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC C:\Users\99836932\.julia\packages\AdvancedHMC\iWHPQ\src\hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC C:\Users\99836932\.julia\packages\AdvancedHMC\iWHPQ\src\hamiltonian.jl:47
┌ Info: Fou

12×8 DataFrame
 Row │ parameters  mean        std       naive_se    mcse        ess       rhat      ess_per_sec 
     │ Symbol      Float64     Float64   Float64     Float64     Float64   Float64   Float64     
─────┼───────────────────────────────────────────────────────────────────────────────────────────
   1 │ α           70.9703     5.1184    0.080929    0.139945    1195.76   1.00102       35.6061
   2 │ σ            6.5362     0.280443  0.00443419  0.00524903  2798.96   1.00061       83.3445
   3 │ τ[1]         6.25541    5.41333   0.0855922   0.214566     619.93   1.00211       18.4596
   4 │ τ[2]         6.27494    5.2265    0.0826383   0.202745     616.858  1.00356       18.3682
   5 │ βⱼ[1,1]      0.235394   0.790277  0.0124954   0.0189687   1634.14   0.999829      48.6598
   6 │ βⱼ[2,1]     -0.89466    1.03596   0.01638     0.0324722   1074.36   1.00128       31.9911
   7 │ βⱼ[3,1]      0.554768   0.860399  0.0136041   0.0239711   1303.5    1.002         38.8144
   8 │ βⱼ[4,

# 3 - Modelo de inclinação de interceptação aleatória

In [14]:
# varying_intercept_slope
@model function varying_intercept_slope(X, idx, y; n_gr=length(unique(idx)), predictors=size(X, 2))
    #priors
    α ~ Normal(mean(y), 2.5 * std(y))                     # population-level intercept
    σ ~ Exponential(1 / std(y))                           # residual SD
    #prior for variance of random intercepts and slopes
    #usually requires thoughtful specification
    τₐ ~ truncated(Cauchy(0, 2); lower=0)                 # group-level SDs intercepts
    τᵦ ~ filldist(truncated(Cauchy(0, 2); lower=0), n_gr) # group-level slopes SDs
    αⱼ ~ filldist(Normal(0, τₐ), n_gr)                    # group-level intercepts
    βⱼ ~ filldist(Normal(0, 1), predictors, n_gr)         # group-level standard normal slopes

    #likelihood
    ŷ = α .+ αⱼ[idx] .+ X * βⱼ * τᵦ
    y ~ MvNormal(ŷ, σ^2 * I)
end;

In [15]:
model_intercept_slope = varying_intercept_slope(X, idx, y)
chain_intercept_slope = sample(model_intercept_slope, NUTS(), MCMCThreads(), 1_000, 4)
summarystats(chain_intercept_slope) |> DataFrame |> println

┌ Warning: Only a single thread available: MCMC chains are not sampled in parallel
└ @ AbstractMCMC C:\Users\99836932\.julia\packages\AbstractMCMC\fnRmh\src\sample.jl:291
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC C:\Users\99836932\.julia\packages\AdvancedHMC\iWHPQ\src\hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC C:\Users\99836932\.julia\packages\AdvancedHMC\iWHPQ\src\hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC C:\Users\99836932\.julia\packages\AdvancedHMC\iWHPQ\src\hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC

15×8 DataFrame
 Row │ parameters  mean        std       naive_se    mcse        ess       rhat      ess_per_sec 
     │ Symbol      Float64     Float64   Float64     Float64     Float64   Float64   Float64     
─────┼───────────────────────────────────────────────────────────────────────────────────────────
   1 │ α           70.7216     6.6562    0.105244    0.181309     977.525  1.00415       19.0439
   2 │ σ            5.8707     0.264046  0.00417494  0.00452719  3197.4    0.999426      62.291
   3 │ τₐ           6.13431    5.6476    0.0892964   0.169549    1028.77   1.00173       20.0424
   4 │ τᵦ[1]        6.10334    5.25169   0.0830364   0.223342     559.866  1.01507       10.9072
   5 │ τᵦ[2]        6.11804    4.9705    0.0785904   0.194656     596.697  1.01336       11.6247
   6 │ αⱼ[1]       -3.4754     4.87065   0.0770118   0.167706     715.271  1.00473       13.9348
   7 │ αⱼ[2]        3.71458    4.88746   0.0772775   0.167296     725.576  1.0045        14.1355
   8 │ βⱼ[1,1